In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [80]:
df = pd.read_csv('data/raw/playstore_reviews.csv')
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it...,1
887,com.rovio.angrybirds,all time legendary game the birthday party le...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad review...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoy...,1


In [81]:

if 'package_name' in df.columns:
    df = df.drop(['package_name'], axis=1)
df["review"] = df["review"].str.strip().str.lower()
df

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0
...,...,...
886,loved it i loooooooooooooovvved it because it ...,1
887,all time legendary game the birthday party lev...,1
888,ads are way to heavy listen to the bad reviews...,0
889,fun works perfectly well. ads aren't as annoyi...,1


In [82]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, stop_words='english')

X = vectorizer.fit_transform(df['review'])
X_train, X_test, y_train, y_test = train_test_split(X, df['polarity'], test_size=0.2, random_state=42)

X_train

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12575 stored elements and shape (712, 3721)>

## BernoulliNB model

In [83]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

In [86]:
model = BernoulliNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7597765363128491
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.91      0.84       126
           1       0.66      0.40      0.49        53

    accuracy                           0.76       179
   macro avg       0.72      0.65      0.67       179
weighted avg       0.74      0.76      0.74       179

